<a href="https://colab.research.google.com/github/guanyaohan/HW1/blob/master/BitFit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import RobertaForSequenceClassification, RobertaTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Load SST2 dataset
print("Loading SST2 dataset...")
dataset = load_dataset("sst2")

# Prepare datasets
print("Preparing datasets...")
train_dataset = dataset["train"]
original_val_dataset = dataset["validation"]

# Split validation dataset into new validation and test sets
val_size = len(original_val_dataset) // 2
val_dataset = Dataset.from_dict(original_val_dataset[:val_size])
test_dataset = Dataset.from_dict(original_val_dataset[val_size:])

print(f"Train dataset size: {len(train_dataset)}")
print(f"New validation dataset size: {len(val_dataset)}")
print(f"New test dataset size: {len(test_dataset)}")

# Load RoBERTa tokenizer and model
print("Loading RoBERTa tokenizer and model...")
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base")

# Implement BitFit: freeze all parameters except biases
for name, param in model.named_parameters():
    if 'bias' not in name:
        param.requires_grad = False

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)

# Process and filter the datasets
def process_dataset(dataset, split_name):
    print(f"Processing {split_name} dataset...")
    tokenized = dataset.map(tokenize_function, batched=True, remove_columns=['sentence', 'idx'])
    tokenized = tokenized.rename_column("label", "labels")
    return tokenized

tokenized_datasets = {
    "train": process_dataset(train_dataset, "train"),
    "validation": process_dataset(val_dataset, "validation"),
    "test": process_dataset(test_dataset, "test")
}

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        if self.args.logging_steps > 0 and self.state.global_step % self.args.logging_steps == 0:
            predictions = torch.argmax(logits, dim=-1)
            accuracy = torch.sum(predictions == labels).item() / len(labels)
            self.log({"train_accuracy": accuracy})

        return (loss, outputs) if return_outputs else loss

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions)
    }

training_args = TrainingArguments(
    output_dir="./results_bitfit",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs_bitfit",
    logging_steps=5,
    evaluation_strategy="steps",
    eval_steps=25,
    save_steps=50,
    load_best_model_at_end=True,
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

print("Starting BitFit training...")
train_result = trainer.train()


train_loss = []
train_acc = []
val_acc = []
steps = []
val_steps = []

last_step = 0
for log in trainer.state.log_history:
    if 'step' in log:
        last_step = log['step']

    if 'loss' in log and 'step' in log:
        steps.append(log['step'])
        train_loss.append(log['loss'])
        if len(train_acc) < len(steps):
            train_acc.append(None)

    if 'train_accuracy' in log:
        if len(steps) > 0 and steps[-1] == last_step:
            train_acc[-1] = log['train_accuracy']
        else:
            steps.append(last_step)
            train_acc.append(log['train_accuracy'])
            train_loss.append(None)

    if 'eval_accuracy' in log and 'step' in log:
        val_steps.append(log['step'])
        val_acc.append(log['eval_accuracy'])

max_length = max(len(steps), len(train_loss), len(train_acc))
steps = steps + [steps[-1]] * (max_length - len(steps))
train_loss = train_loss + [None] * (max_length - len(train_loss))
train_acc = train_acc + [None] * (max_length - len(train_acc))

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 15), sharex=True)

ax1.plot([step for step, loss in zip(steps, train_loss) if loss is not None],
         [loss for loss in train_loss if loss is not None],
         label='Train Loss', color='blue')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss (BitFit)')
ax1.legend()

ax2.plot([step for step, acc in zip(steps, train_acc) if acc is not None],
         [acc for acc in train_acc if acc is not None],
         label='Train Accuracy', color='green')
ax2.set_ylabel('Accuracy')
ax2.set_title('Training Accuracy (BitFit)')
ax2.legend()

ax3.plot(val_steps, val_acc, label='Validation Accuracy', color='red')
ax3.set_ylabel('Accuracy')
ax3.set_title('Validation Accuracy (BitFit)')
ax3.legend()

ax3.set_xlabel('Steps')
plt.tight_layout()
plt.savefig('bitfit_training_metrics.png')
plt.close()

print("BitFit training completed. Metrics plot saved as 'bitfit_training_metrics.png'.")

# Evaluate on test set
test_results = trainer.evaluate(tokenized_datasets["test"])
print(f"Test set results: {test_results}")

Loading SST2 dataset...
Preparing datasets...
Train dataset size: 67349
New validation dataset size: 436
New test dataset size: 436
Loading RoBERTa tokenizer and model...


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing train dataset...
Processing validation dataset...


Map:   0%|          | 0/436 [00:00<?, ? examples/s]

Processing test dataset...


...

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Starting BitFit training...


Step,Training Loss,Validation Loss,Accuracy
25,0.711300,0.697172,0.477064
50,0.700200,0.696407,0.477064
75,0.701000,0.695623,0.477064
100,0.699200,0.694355,0.477064
125,0.692600,0.692862,0.527523
150,0.691600,0.692542,0.522936
175,0.692400,0.692291,0.522936
200,0.689800,0.692028,0.522936
225,0.677900,0.691905,0.522936
250,0.693100,0.691939,0.522936


BitFit training completed. Metrics plot saved as 'bitfit_training_metrics.png'.


Test set results: {'eval_loss': 0.35327598452568054, 'eval_accuracy': 0.8600917431192661, 'eval_runtime': 0.7973, 'eval_samples_per_second': 546.836, 'eval_steps_per_second': 8.779, 'epoch': 1.0}
